In [ ]:
import json
import os

from dotenv import load_dotenv

from pydantic import BaseModel, ValidationError, Field
from typing import List

from openai import AsyncOpenAI

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

from semantic_kernel.functions import KernelArguments

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = OpenAIChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [ ]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

# महत्त्वपूर्ण नियमहरू

## परिचय
यो दस्तावेजले तपाईंलाई परियोजनाको लागि आवश्यक महत्त्वपूर्ण नियमहरू बुझ्न मद्दत गर्दछ। कृपया यी नियमहरू पालना गर्नुहोस्।

## नियमहरू

### १. कोडिङ मानकहरू
- कोड स्पष्ट र पढ्न सजिलो हुनुपर्छ।
- सबै परिवर्तनहरू राम्रोसँग टिप्पणी गरिनुपर्छ।
- नामकरण कन्वेन्सनहरू पालना गर्नुहोस्, जस्तै `camelCase` वा `snake_case`।

### २. फाइल संरचना
- सबै फाइलहरू उचित फोल्डरमा राख्नुपर्छ।
- फाइल नामहरू वर्णनात्मक र सान्दर्भिक हुनुपर्छ।

### ३. परीक्षण
- सबै नयाँ फिचरहरूका लागि परीक्षणहरू लेख्नुपर्छ।
- परीक्षणहरू स्वचालित रूपमा चल्नुपर्ने सुनिश्चित गर्नुहोस्।

### ४. कोड समीक्षा
- सबै कोड परिवर्तनहरू समीक्षा गर्नुपर्छ।
- कोड समीक्षा गर्दा रचनात्मक प्रतिक्रिया दिनुहोस्।

[!NOTE] यी नियमहरू पालना गर्दा परियोजनाको गुणस्तर उच्च रहन्छ।

## निष्कर्ष
यी नियमहरू पालना गरेर, हामी एक राम्रो र व्यवस्थित परियोजना निर्माण गर्न सक्छौं।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
